# LAB 3: PROMPT FLOW & ORCHESTRATION - COMPLETE GUIDE

**Course:** Advanced Prompt Engineering Training  
**Session:** Session 3 - RAG & Advanced Retrieval (Day 2)  
**Duration:** 120 minutes (2 hours)  
**Type:** Comprehensive Orchestration Workshop

## LAB OVERVIEW

This comprehensive lab teaches you to build **production-grade orchestrated AI workflows**. You'll progress through four interconnected modules:

1. **Sequential Chains** - Linear multi-step workflows
2. **Parallel Processing** - Concurrent execution for speed
3. **Conditional Workflows** - Dynamic routing based on data
4. **Production Orchestration** - Error handling, monitoring, deployment

**Scenario:** You're building an AI-powered loan processing system for a bank. Starting with simple sequential workflows, you'll evolve to a production-ready system handling thousands of applications daily.

### Step 1: Import Libraries

In [ ]:
# Lab 3: Prompt Flow & Orchestration
# Advanced Prompt Engineering Training - Session 3

import os
import json
import time
import re
from datetime import datetime
from typing import List, Dict, Tuple, Optional, Any, Callable
from dataclasses import dataclass, field
from concurrent.futures import ThreadPoolExecutor, as_completed
from enum import Enum
import traceback

from openai import OpenAI
import pandas as pd
import numpy as np

from dotenv import load_dotenv

load_dotenv(override=True)

print("✓ Libraries imported successfully")

### Step 2: Configure OpenAI Client

In [ ]:
# Initialize OpenAI client
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# Model configurations
GPT4 = os.environ.get("MODEL_NAME", "gpt-4o")
GPT35 = os.environ.get("FAST_MODEL_NAME", "gpt-3.5-turbo")

# Default settings
DEFAULT_TEMPERATURE = 0
DEFAULT_MAX_TOKENS = 2000

print(f"✓ OpenAI client configured")
print(f"✓ Primary model: {GPT4}")
print(f"✓ Fast model: {GPT35}")

### Step 3: Create Core Helper Functions

In [ ]:
@dataclass
class LLMResponse:
    """Standard response format for all LLM calls"""
    content: str
    model: str
    tokens: int
    latency: float
    timestamp: str = field(default_factory=lambda: datetime.now().isoformat())
    
    def to_dict(self) -> Dict:
        return {
            "content": self.content,
            "model": self.model,
            "tokens": self.tokens,
            "latency": self.latency,
            "timestamp": self.timestamp
        }

def call_llm(
    prompt: str,
    system_prompt: str = "You are a helpful AI assistant.",
    model: str = GPT4,
    temperature: float = DEFAULT_TEMPERATURE,
    max_tokens: int = DEFAULT_MAX_TOKENS
) -> LLMResponse:
    """
    Unified LLM calling function with metrics tracking
    
    Args:
        prompt: User prompt
        system_prompt: System instructions
        model: Model to use (GPT4 or GPT35)
        temperature: Sampling temperature
        max_tokens: Maximum response tokens
    
    Returns:
        LLMResponse with content and metadata
    """
    start_time = time.time()
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            temperature=temperature,
            max_tokens=max_tokens
        )
        
        latency = time.time() - start_time
        
        return LLMResponse(
            content=response.choices[0].message.content,
            model=model,
            tokens=response.usage.total_tokens,
            latency=latency
        )
        
    except Exception as e:
        latency = time.time() - start_time
        return LLMResponse(
            content=f"Error: {str(e)}",
            model=model,
            tokens=0,
            latency=latency
        )

def print_response(response: LLMResponse, title: str = "Response"):
    """Pretty print LLM response with metrics"""
    print(f"\n{'='*80}")
    print(f"{title}")
    print(f"{'='*80}")
    print(f"Model: {response.model}")
    print(f"Latency: {response.latency:.2f}s")
    print(f"Tokens: {response.tokens}")
    print(f"\nContent:\n{response.content}")
    print(f"{'='*80}\n")

# Test the setup
test_response = call_llm("Say 'Setup complete!' if you receive this.", model=GPT35)
print_response(test_response, "Setup Test")

### Step 4: Load Sample Data

In [ ]:
# Sample loan application data for the lab
SAMPLE_LOAN_APPLICATION = """
LOAN APPLICATION #LA-2024-5821

Applicant Information:
- Name: Sarah Johnson
- Age: 34
- Employment: Senior Software Engineer at TechCorp
- Annual Income: $145,000
- Employment Duration: 6 years
- Credit Score: 750

Loan Details:
- Loan Type: Home Mortgage
- Loan Amount: $485,000
- Property Value: $620,000
- Down Payment: $135,000 (22%)
- Loan Term: 30 years

Financial Information:
- Monthly Gross Income: $12,083
- Existing Debts: 
  * Car Loan: $425/month (12 months remaining)
  * Student Loan: $280/month (24 months remaining)
  * Credit Card: $150/month average
- Total Monthly Debt: $855

Property Information:
- Property Address: 245 Oak Street, San Francisco, CA
- Property Type: Single-family home
- Year Built: 2018
- Square Footage: 2,100 sq ft
- Condition: Excellent

Supporting Documents:
- 2 years tax returns: Provided
- 3 months pay stubs: Provided
- Bank statements: Provided
- Employment verification: Verified
- Property appraisal: $620,000
"""

# Additional test applications for later exercises
SMALL_LOAN_APP = """
LOAN APPLICATION #LA-2024-5822
Applicant: Mike Chen
Loan Type: Personal Loan
Amount: $15,000
Income: $65,000
Credit Score: 720
Employment: 3 years
"""

LARGE_LOAN_APP = """
LOAN APPLICATION #LA-2024-5823
Applicant: Jennifer Martinez
Loan Type: Commercial Real Estate
Amount: $2,500,000
Business Revenue: $8,000,000/year
Credit Score: 780
Business Age: 12 years
Collateral: Commercial property valued at $3,200,000
"""

print("✓ Sample data loaded")
print(f"✓ Main application: {SAMPLE_LOAN_APPLICATION.split()[3]}")
print(f"✓ Test applications: 2 additional scenarios ready")

---

## PART 1: SEQUENTIAL CHAINS (Lab 3.1)

**Duration:** 30 minutes  
**Objective:** Build linear multi-step workflows where each step depends on the previous

### Theory: Sequential Chain Pattern

**Concept:** Output of Step N becomes input to Step N+1

```
Input → [Step 1] → Result1 → [Step 2] → Result2 → [Step 3] → Final Output
```

**When to use:**
- Steps have clear dependencies
- Linear workflow makes sense
- Need to inspect intermediate results

### Challenge 1.1: Basic Sequential Chain (10 minutes)

**Scenario:** Process a loan application through 3 steps:
1. Extract key information
2. Calculate financial ratios
3. Generate summary

In [ ]:
# Step 1: Extract Information
def step1_extract_info(application: str) -> LLMResponse:
    """Extract key applicant and loan information"""
    
    prompt = f"""
    Extract the following information from this loan application and return as JSON:
    
    Required fields:
    - applicant_name
    - loan_amount (number only)
    - annual_income (number only)
    - credit_score (number only)
    - monthly_debt (total monthly debt payments, number only)
    - loan_type
    - property_value (if applicable, number only)
    - down_payment (if applicable, number only)
    
    Loan Application:
    {application}
    
    Return ONLY valid JSON, no other text.
    """
    
    system_prompt = "You are a data extraction specialist. Always return valid JSON."
    
    return call_llm(prompt, system_prompt, model=GPT35)

In [ ]:
# Step 2: Calculate Financial Ratios
def step2_calculate_ratios(extracted_data: str) -> LLMResponse:
    """Calculate financial ratios from extracted data"""
    
    prompt = f"""
    Given this extracted loan application data, calculate these financial metrics:
    
    Extracted Data:
    {extracted_data}
    
    Calculate:
    1. DTI (Debt-to-Income Ratio): (monthly_debt / monthly_income) × 100
       - monthly_income = annual_income / 12
       - Express as percentage
    
    2. LTV (Loan-to-Value Ratio): (loan_amount / property_value) × 100
       - If property_value exists
       - Express as percentage
    
    3. Estimated Monthly Payment: Rough estimate for 30-year mortgage at 7% interest
       - Use formula: P = L[c(1 + c)^n]/[(1 + c)^n - 1]
       - Where L = loan amount, c = monthly interest rate, n = number of payments
    
    Return as JSON with:
    {{
        "dti_ratio": <percentage>,
        "dti_assessment": "<Good/Fair/Poor based on: <28% = Good, 28-36% = Fair, >36% = Poor>",
        "ltv_ratio": <percentage or null>,
        "ltv_assessment": "<Good/Fair/Poor based on: <80% = Good, 80-90% = Fair, >90% = Poor>",
        "estimated_monthly_payment": <amount>,
        "affordability_check": "<Can afford based on 28% housing ratio rule>"
    }}
    
    Return ONLY valid JSON.
    """
    
    system_prompt = "You are a financial analyst. Always return valid JSON with accurate calculations."
    
    return call_llm(prompt, system_prompt, model=GPT35)

In [ ]:
# Step 3: Generate Executive Summary
def step3_generate_summary(extracted_data: str, ratios: str) -> LLMResponse:
    """Generate executive summary with recommendation"""
    
    prompt = f"""
    Create an executive summary for this loan application.
    
    Applicant Data:
    {extracted_data}
    
    Financial Analysis:
    {ratios}
    
    Write a concise executive summary (3-4 paragraphs) covering:
    
    1. Applicant Overview
       - Name, income, credit score
       - Employment stability
    
    2. Financial Strength
       - DTI and LTV ratios
       - Overall financial health
    
    3. Risk Assessment
       - Key strengths
       - Potential concerns
    
    4. Recommendation
       - APPROVE: Strong application, low risk
       - REVIEW: Acceptable but needs senior review
       - DENY: High risk, does not meet criteria
    
    Be professional and concise. Focus on facts and ratios.
    """
    
    system_prompt = "You are a senior loan officer writing executive summaries for the credit committee."
    
    return call_llm(prompt, system_prompt, model=GPT4)

In [ ]:
# Execute the 3-Step Sequential Chain
def sequential_chain_basic(application: str) -> Dict[str, LLMResponse]:
    """
    Execute 3-step sequential loan processing chain
    
    Returns:
        Dictionary with results from each step plus total metrics
    """
    start_time = time.time()
    results = {}
    
    print("Starting Sequential Chain...")
    print("-" * 80)
    
    # Step 1: Extract Information
    print("\n[1/3] Extracting information...")
    step1_result = step1_extract_info(application)
    results['step1_extraction'] = step1_result
    print(f"✓ Completed in {step1_result.latency:.2f}s ({step1_result.tokens} tokens)")
    
    # Step 2: Calculate Ratios (uses Step 1 output)
    print("\n[2/3] Calculating financial ratios...")
    step2_result = step2_calculate_ratios(step1_result.content)
    results['step2_ratios'] = step2_result
    print(f"✓ Completed in {step2_result.latency:.2f}s ({step2_result.tokens} tokens)")
    
    # Step 3: Generate Summary (uses Step 1 and Step 2 outputs)
    print("\n[3/3] Generating executive summary...")
    step3_result = step3_generate_summary(step1_result.content, step2_result.content)
    results['step3_summary'] = step3_result
    print(f"✓ Completed in {step3_result.latency:.2f}s ({step3_result.tokens} tokens)")
    
    # Calculate total metrics
    total_time = time.time() - start_time
    total_tokens = sum(r.tokens for r in results.values())
    
    print("\n" + "=" * 80)
    print("SEQUENTIAL CHAIN COMPLETE")
    print("=" * 80)
    print(f"Total Time: {total_time:.2f}s")
    print(f"Total Tokens: {total_tokens}")
    print(f"Steps Executed: {len(results)}")
    print("=" * 80)
    
    # Store totals
    results['_metadata'] = {
        'total_time': total_time,
        'total_tokens': total_tokens,
        'steps': len(results) - 1
    }
    
    return results


# Execute the chain
print("Testing Sequential Chain with Sample Loan Application\n")
results = sequential_chain_basic(SAMPLE_LOAN_APPLICATION)

# Display final summary
print("\n" + "="*80)
print("FINAL EXECUTIVE SUMMARY")
print("="*80)
print(results['step3_summary'].content)
print("="*80)

### Challenge 1.2: Enhanced Sequential Chain with State (10 minutes)

**Objective:** Add state management and intermediate result inspection

In [ ]:
@dataclass
class LoanProcessingContext:
    """Context object that accumulates state across workflow"""
    application_id: str
    application_text: str
    extracted_data: Optional[Dict] = None
    financial_ratios: Optional[Dict] = None
    risk_assessment: Optional[str] = None
    recommendation: Optional[str] = None
    processing_log: List[str] = field(default_factory=list)
    total_tokens: int = 0
    total_latency: float = 0.0
    
    def log(self, message: str):
        """Add timestamped log entry"""
        timestamp = datetime.now().strftime("%H:%M:%S.%f")[:-3]
        self.processing_log.append(f"[{timestamp}] {message}")
        print(f"[{timestamp}] {message}")
    
    def add_metrics(self, response: LLMResponse):
        """Track cumulative metrics"""
        self.total_tokens += response.tokens
        self.total_latency += response.latency
    
    def to_dict(self) -> Dict:
        """Convert to dictionary"""
        return {
            "application_id": self.application_id,
            "extracted_data": self.extracted_data,
            "financial_ratios": self.financial_ratios,
            "risk_assessment": self.risk_assessment,
            "recommendation": self.recommendation,
            "total_tokens": self.total_tokens,
            "total_latency": self.total_latency,
            "processing_log": self.processing_log
        }

In [ ]:
def stateful_sequential_workflow(application: str, app_id: str) -> LoanProcessingContext:
    """
    Process loan application with full state tracking and error handling
    """
    context = LoanProcessingContext(
        application_id=app_id,
        application_text=application
    )
    
    context.log(f"Started processing application {app_id}")
    
    try:
        # Step 1: Extract Data
        context.log("Step 1/3: Extracting information...")
        response1 = step1_extract_info(application)
        context.add_metrics(response1)
        
        # Parse and store extracted data
        try:
            context.extracted_data = json.loads(response1.content)
            context.log(f"✓ Extracted data for {context.extracted_data.get('applicant_name', 'Unknown')}")
        except json.JSONDecodeError:
            context.log("⚠ Warning: Could not parse extraction JSON, using raw content")
            context.extracted_data = {"raw": response1.content}
        
        # Step 2: Calculate Ratios
        context.log("Step 2/3: Calculating financial ratios...")
        response2 = step2_calculate_ratios(response1.content)
        context.add_metrics(response2)
        
        # Parse and store ratios
        try:
            context.financial_ratios = json.loads(response2.content)
            dti = context.financial_ratios.get('dti_ratio', 'N/A')
            context.log(f"✓ Calculated DTI: {dti}%")
        except json.JSONDecodeError:
            context.log("⚠ Warning: Could not parse ratios JSON, using raw content")
            context.financial_ratios = {"raw": response2.content}
        
        # Step 3: Generate Recommendation
        context.log("Step 3/3: Generating recommendation...")
        response3 = step3_generate_summary(response1.content, response2.content)
        context.add_metrics(response3)
        
        context.risk_assessment = response3.content
        
        # Extract recommendation from summary
        content_upper = response3.content.upper()
        if "RECOMMEND: APPROVE" in content_upper or "RECOMMENDATION: APPROVE" in content_upper:
            context.recommendation = "APPROVE"
        elif "RECOMMEND: DENY" in content_upper or "RECOMMENDATION: DENY" in content_upper:
            context.recommendation = "DENY"
        else:
            context.recommendation = "REVIEW"
        
        context.log(f"✓ Recommendation: {context.recommendation}")
        context.log(f"Processing complete - {context.total_tokens} tokens, {context.total_latency:.2f}s")
        
    except Exception as e:
        context.log(f"✗ Error during processing: {str(e)}")
        context.recommendation = "ERROR"
    
    return context


# Test stateful workflow
print("="*80)
print("STATEFUL SEQUENTIAL WORKFLOW TEST")
print("="*80 + "\n")

context = stateful_sequential_workflow(SAMPLE_LOAN_APPLICATION, "LA-2024-5821")

# Display results
print("\n" + "="*80)
print("PROCESSING CONTEXT SUMMARY")
print("="*80)
print(f"Application ID: {context.application_id}")
print(f"Applicant: {context.extracted_data.get('applicant_name', 'N/A') if context.extracted_data else 'N/A'}")
print(f"Recommendation: {context.recommendation}")
print(f"Total Tokens: {context.total_tokens}")
print(f"Total Latency: {context.total_latency:.2f}s")
print("\nProcessing Log:")
for log_entry in context.processing_log:
    print(f"  {log_entry}")
print("="*80)

---

## PART 2: PARALLEL PROCESSING (Lab 3.2)

**Duration:** 30 minutes  
**Objective:** Execute independent tasks concurrently to reduce latency

### Theory: Parallel Pattern

**Concept:** Fork into independent tasks, execute concurrently, then join results

```
Input → Fork → [Task A]
              [Task B] → Join → Synthesize → Output
              [Task C]
```

**When to use:**
- Tasks are independent (no dependencies)
- Speed/latency matters
- Can afford higher compute costs

**Expected speedup:** ~3x for 3 parallel tasks

### Challenge 2.1: Parallel Credit Checks (15 minutes)

**Scenario:** Verify loan application across 3 independent sources:
1. Credit bureau check
2. Employment verification
3. Identity verification

All can run in parallel!

In [ ]:
def check_credit_bureau(applicant_data: Dict) -> LLMResponse:
    """Simulate credit bureau verification"""
    
    prompt = f"""
    Perform credit bureau verification for this applicant:
    
    Applicant Data:
    {json.dumps(applicant_data, indent=2)}
    
    Verify and return JSON with:
    {{
        "credit_score": <number>,
        "credit_score_valid": <true/false>,
        "payment_history": "<Excellent/Good/Fair/Poor>",
        "outstanding_debts": <total amount>,
        "bankruptcy_history": <true/false>,
        "verification_status": "VERIFIED" or "FAILED",
        "notes": "<any concerns or all clear>"
    }}
    
    Simulate realistic credit bureau response. Use the credit score from applicant_data.
    """
    
    system_prompt = "You are a credit bureau API returning verification results."
    
    return call_llm(prompt, system_prompt, model=GPT35)


def verify_employment(applicant_data: Dict) -> LLMResponse:
    """Simulate employment verification"""
    
    prompt = f"""
    Verify employment for this applicant:
    
    Applicant Data:
    {json.dumps(applicant_data, indent=2)}
    
    Return JSON with:
    {{
        "employer": "<company name>",
        "position": "<job title>",
        "employment_verified": <true/false>,
        "tenure_years": <number>,
        "annual_income_verified": <number>,
        "income_matches_stated": <true/false>,
        "employment_stable": <true/false>,
        "verification_status": "VERIFIED" or "FAILED",
        "notes": "<any discrepancies or all clear>"
    }}
    
    Use data from applicant_data. Simulate realistic employment verification.
    """
    
    system_prompt = "You are an employment verification service API."
    
    return call_llm(prompt, system_prompt, model=GPT35)


def verify_identity(applicant_data: Dict) -> LLMResponse:
    """Simulate identity verification"""
    
    prompt = f"""
    Verify identity for this applicant:
    
    Applicant Data:
    {json.dumps(applicant_data, indent=2)}
    
    Return JSON with:
    {{
        "identity_verified": <true/false>,
        "name_matches": <true/false>,
        "address_verified": <true/false>,
        "fraud_flags": <number of flags, 0 = none>,
        "risk_level": "<LOW/MEDIUM/HIGH>",
        "verification_status": "VERIFIED" or "FAILED",
        "notes": "<any red flags or all clear>"
    }}
    
    Simulate realistic identity verification check.
    """
    
    system_prompt = "You are an identity verification service API."
    
    return call_llm(prompt, system_prompt, model=GPT35)

In [ ]:
def parallel_verification_checks(applicant_data: Dict) -> Dict[str, Any]:
    """
    Execute all three verification checks in parallel
    
    Returns:
        Dictionary with results from each check and performance metrics
    """
    start_time = time.time()
    results = {}
    
    print("Starting Parallel Verification Checks...")
    print("-" * 80)
    
    # Define tasks
    tasks = {
        'credit_bureau': check_credit_bureau,
        'employment': verify_employment,
        'identity': verify_identity
    }
    
    # Execute in parallel using ThreadPoolExecutor
    with ThreadPoolExecutor(max_workers=3) as executor:
        # Submit all tasks
        future_to_name = {
            executor.submit(func, applicant_data): name 
            for name, func in tasks.items()
        }
        
        # Collect results as they complete
        for future in as_completed(future_to_name):
            task_name = future_to_name[future]
            try:
                result = future.result()
                results[task_name] = result
                print(f"✓ {task_name.replace('_', ' ').title()}: {result.latency:.2f}s ({result.tokens} tokens)")
            except Exception as e:
                print(f"✗ {task_name} failed: {str(e)}")
                results[task_name] = LLMResponse(f"Error: {str(e)}", GPT35, 0, 0.0)
    
    # Calculate metrics
    total_time = time.time() - start_time
    total_tokens = sum(r.tokens for r in results.values() if isinstance(r, LLMResponse))
    sequential_time = sum(r.latency for r in results.values() if isinstance(r, LLMResponse))
    speedup = sequential_time / total_time if total_time > 0 else 0
    
    print("\n" + "=" * 80)
    print("PARALLEL VERIFICATION COMPLETE")
    print("=" * 80)
    print(f"Wall Clock Time: {total_time:.2f}s")
    print(f"Sequential Time (if run in series): {sequential_time:.2f}s")
    print(f"Speedup: {speedup:.2f}x")
    print(f"Total Tokens: {total_tokens}")
    print("=" * 80)
    
    results['_metadata'] = {
        'wall_clock_time': total_time,
        'sequential_time': sequential_time,
        'speedup': speedup,
        'total_tokens': total_tokens
    }
    
    return results


# Test parallel verification
test_applicant = {
    "name": "Sarah Johnson",
    "annual_income": 145000,
    "credit_score": 750,
    "employer": "TechCorp",
    "position": "Senior Software Engineer",
    "employment_years": 6,
    "address": "245 Oak Street, San Francisco, CA"
}

verification_results = parallel_verification_checks(test_applicant)

# Parse and display verification results
print("\n" + "="*80)
print("VERIFICATION RESULTS SUMMARY")
print("="*80)

for check_name, result in verification_results.items():
    if check_name != '_metadata':
        print(f"\n{check_name.replace('_', ' ').title()}:")
        try:
            parsed = json.loads(result.content)
            for key, value in parsed.items():
                print(f"  {key}: {value}")
        except:
            print(f"  {result.content[:200]}...")

print("="*80)

### Challenge 2.2: Combine Sequential + Parallel (15 minutes)

**Objective:** Build hybrid workflow combining both patterns

```
Step 1: Extract data (sequential)
    ↓
Step 2: Fork into parallel checks → Join
    ↓
Step 3: Synthesize results (sequential)
```

In [ ]:
def hybrid_loan_processing(application: str, app_id: str) -> Dict[str, Any]:
    """
    Comprehensive loan processing combining sequential and parallel patterns
    
    Workflow:
    1. Sequential: Extract applicant data
    2. Parallel: Run verification checks
    3. Sequential: Synthesize final decision
    """
    overall_start = time.time()
    results = {}
    
    print("="*80)
    print(f"HYBRID WORKFLOW: Application {app_id}")
    print("="*80 + "\n")
    
    # STEP 1: Extract Data (Sequential - must happen first)
    print("[Step 1/3] Extracting applicant data...")
    step1_start = time.time()
    
    extraction = step1_extract_info(application)
    try:
        applicant_data = json.loads(extraction.content)
    except:
        applicant_data = {"error": "Could not parse extraction"}
    
    step1_time = time.time() - step1_start
    results['extraction'] = extraction
    print(f"✓ Extraction complete: {step1_time:.2f}s\n")
    
    # STEP 2: Parallel Verification Checks
    print("[Step 2/3] Running parallel verification checks...")
    step2_start = time.time()
    
    verification_results = parallel_verification_checks(applicant_data)
    
    step2_time = time.time() - step2_start
    results['verifications'] = verification_results
    print(f"\n✓ Verifications complete: {step2_time:.2f}s\n")
    
    # STEP 3: Synthesize Final Decision (Sequential - needs all previous data)
    print("[Step 3/3] Synthesizing final decision...")
    step3_start = time.time()
    
    synthesis_prompt = f"""
    Based on the following loan application data and verification results, 
    provide a final credit decision.
    
    Applicant Data:
    {json.dumps(applicant_data, indent=2)}
    
    Verification Results:
    - Credit Bureau: {verification_results.get('credit_bureau', LLMResponse('', '', 0, 0)).content[:500]}
    - Employment: {verification_results.get('employment', LLMResponse('', '', 0, 0)).content[:500]}
    - Identity: {verification_results.get('identity', LLMResponse('', '', 0, 0)).content[:500]}
    
    Provide final decision in this format:
    
    DECISION: [APPROVE/DENY/MANUAL_REVIEW]
    
    CONFIDENCE: [HIGH/MEDIUM/LOW]
    
    KEY FACTORS:
    - [List 3-5 key factors influencing this decision]
    
    REASONING:
    [2-3 sentences explaining the decision]
    
    CONDITIONS (if applicable):
    [Any conditions for approval, or N/A]
    """
    
    final_decision = call_llm(synthesis_prompt, 
                             "You are a senior credit officer making final loan decisions.",
                             model=GPT4)
    
    step3_time = time.time() - step3_start
    results['final_decision'] = final_decision
    print(f"✓ Decision complete: {step3_time:.2f}s\n")
    
    # Calculate overall metrics
    total_time = time.time() - overall_start
    total_tokens = (extraction.tokens + 
                   verification_results['_metadata']['total_tokens'] + 
                   final_decision.tokens)
    
    # Estimate sequential time (if everything ran in sequence)
    est_sequential = (step1_time + 
                     verification_results['_metadata']['sequential_time'] + 
                     step3_time)
    
    speedup = est_sequential / total_time
    
    print("="*80)
    print("HYBRID WORKFLOW COMPLETE")
    print("="*80)
    print(f"Total Wall Clock Time: {total_time:.2f}s")
    print(f"Estimated Sequential Time: {est_sequential:.2f}s")
    print(f"Overall Speedup: {speedup:.2f}x")
    print(f"Total Tokens: {total_tokens}")
    print("\nTiming Breakdown:")
    print(f"  Step 1 (Sequential): {step1_time:.2f}s")
    print(f"  Step 2 (Parallel): {step2_time:.2f}s")
    print(f"  Step 3 (Sequential): {step3_time:.2f}s")
    print("="*80)
    
    results['_metadata'] = {
        'total_time': total_time,
        'estimated_sequential_time': est_sequential,
        'speedup': speedup,
        'total_tokens': total_tokens,
        'step_times': {
            'extraction': step1_time,
            'verification': step2_time,
            'decision': step3_time
        }
    }
    
    return results


# Execute hybrid workflow
hybrid_results = hybrid_loan_processing(SAMPLE_LOAN_APPLICATION, "LA-2024-5821")

# Display final decision
print("\n" + "="*80)
print("FINAL CREDIT DECISION")
print("="*80)
print(hybrid_results['final_decision'].content)
print("="*80)

---

## PART 3: CONDITIONAL WORKFLOWS (Lab 3.3)

**Duration:** 30 minutes  
**Objective:** Build dynamic workflows that route based on data

### Theory: Conditional Pattern

**Concept:** Different inputs take different processing paths

```
Input → [Classify] → Decision Point
                         ├─ Path A (simple processing)
                         ├─ Path B (standard processing)
                         └─ Path C (enhanced processing)
```

**When to use:**
- Different inputs need different handling
- Want to optimize processing (don't over-process simple cases)
- Clear decision criteria exist

### Challenge 3.1: Risk-Based Routing (20 minutes)

**Scenario:** Route loans to different approval workflows based on amount:
- **Small loans** (<$100K): Fast-track approval (2 steps)
- **Medium loans** ($100K-$500K): Standard review (4 steps)
- **Large loans** (>$500K): Enhanced due diligence (6 steps)

In [ ]:
class LoanRouter:
    """Intelligent loan routing based on amount and risk"""
    
    SMALL_LOAN_THRESHOLD = 100000
    LARGE_LOAN_THRESHOLD = 500000
    
    @staticmethod
    def classify_loan(loan_amount: float) -> str:
        """Classify loan into size category"""
        if loan_amount < LoanRouter.SMALL_LOAN_THRESHOLD:
            return "SMALL"
        elif loan_amount < LoanRouter.LARGE_LOAN_THRESHOLD:
            return "MEDIUM"
        else:
            return "LARGE"
    
    @staticmethod
    def process_small_loan(application: str, loan_amount: float) -> Dict:
        """Fast-track processing for small loans (<$100K)"""
        print(f"\n{'='*80}")
        print(f"SMALL LOAN FAST-TRACK WORKFLOW (${loan_amount:,.0f})")
        print(f"{'='*80}\n")
        
        start_time = time.time()
        results = {}
        
        # Step 1: Quick credit check
        print("[1/2] Quick credit assessment...")
        step1_prompt = f"""
        Perform quick credit assessment for small loan:
        
        Application:
        {application}
        
        Provide quick decision based on:
        - Credit score (must be >650)
        - Income stability
        - Basic debt check
        
        Return JSON:
        {{
            "credit_acceptable": <true/false>,
            "quick_decision": "<APPROVE/DENY/NEEDS_REVIEW>",
            "reason": "<brief explanation>"
        }}
        """
        
        credit_check = call_llm(step1_prompt, 
                               "You are a fast-track credit assessor for small loans.",
                               model=GPT35)
        results['credit_check'] = credit_check
        print(f"✓ Credit check complete ({credit_check.latency:.2f}s)")
        
        # Step 2: Auto-decision
        print("[2/2] Generating auto-decision...")
        try:
            credit_data = json.loads(credit_check.content)
            decision = credit_data.get('quick_decision', 'NEEDS_REVIEW')
        except:
            decision = "NEEDS_REVIEW"
        
        results['final_decision'] = decision
        results['workflow_type'] = "FAST_TRACK"
        results['processing_time'] = time.time() - start_time
        
        print(f"✓ Decision: {decision}")
        print(f"\nTotal processing time: {results['processing_time']:.2f}s")
        print(f"{'='*80}\n")
        
        return results
    
    @staticmethod
    def process_medium_loan(application: str, loan_amount: float) -> Dict:
        """Standard processing for medium loans ($100K-$500K)"""
        print(f"\n{'='*80}")
        print(f"MEDIUM LOAN STANDARD WORKFLOW (${loan_amount:,.0f})")
        print(f"{'='*80}\n")
        
        start_time = time.time()
        results = {}
        
        # Step 1: Extract data
        print("[1/4] Extracting application data...")
        extraction = step1_extract_info(application)
        results['extraction'] = extraction
        print(f"✓ Complete ({extraction.latency:.2f}s)")
        
        # Step 2: Credit & Employment (parallel)
        print("[2/4] Running credit and employment verification...")
        try:
            applicant_data = json.loads(extraction.content)
        except:
            applicant_data = {}
        
        verification_start = time.time()
        with ThreadPoolExecutor(max_workers=2) as executor:
            credit_future = executor.submit(check_credit_bureau, applicant_data)
            employment_future = executor.submit(verify_employment, applicant_data)
            
            credit_result = credit_future.result()
            employment_result = employment_future.result()
        
        verification_time = time.time() - verification_start
        results['verification'] = {
            'credit': credit_result,
            'employment': employment_result,
            'time': verification_time
        }
        print(f"✓ Verifications complete ({verification_time:.2f}s)")
        
        # Step 3: Calculate ratios
        print("[3/4] Calculating financial ratios...")
        ratios = step2_calculate_ratios(extraction.content)
        results['ratios'] = ratios
        print(f"✓ Complete ({ratios.latency:.2f}s)")
        
        # Step 4: Generate recommendation
        print("[4/4] Generating recommendation...")
        recommendation = step3_generate_summary(extraction.content, ratios.content)
        results['recommendation'] = recommendation
        print(f"✓ Complete ({recommendation.latency:.2f}s)")
        
        results['workflow_type'] = "STANDARD"
        results['processing_time'] = time.time() - start_time
        
        print(f"\nTotal processing time: {results['processing_time']:.2f}s")
        print(f"{'='*80}\n")
        
        return results
    
    @staticmethod
    def process_large_loan(application: str, loan_amount: float) -> Dict:
        """Enhanced due diligence for large loans (>$500K)"""
        print(f"\n{'='*80}")
        print(f"LARGE LOAN ENHANCED DUE DILIGENCE (${loan_amount:,.0f})")
        print(f"{'='*80}\n")
        
        start_time = time.time()
        results = {}
        
        # Step 1: Extract comprehensive data
        print("[1/6] Comprehensive data extraction...")
        extraction = step1_extract_info(application)
        results['extraction'] = extraction
        print(f"✓ Complete ({extraction.latency:.2f}s)")
        
        # Step 2: Full parallel verifications
        print("[2/6] Running comprehensive verification suite...")
        try:
            applicant_data = json.loads(extraction.content)
        except:
            applicant_data = {}
        
        full_verification = parallel_verification_checks(applicant_data)
        results['verification'] = full_verification
        print(f"✓ All verifications complete")
        
        # Step 3: Advanced financial analysis
        print("[3/6] Advanced financial analysis...")
        analysis_prompt = f"""
        Perform advanced financial analysis for large commercial loan:
        
        Application Data:
        {extraction.content}
        
        Provide comprehensive analysis including:
        - All standard ratios (DTI, LTV, DSCR)
        - Cash flow analysis
        - Collateral assessment
        - Market risk factors
        - Stress testing scenarios
        
        Return detailed JSON with all metrics and assessments.
        """
        
        financial_analysis = call_llm(analysis_prompt,
                                     "You are a senior financial analyst for large loans.",
                                     model=GPT4)
        results['financial_analysis'] = financial_analysis
        print(f"✓ Complete ({financial_analysis.latency:.2f}s)")
        
        # Step 4: Risk modeling
        print("[4/6] Risk modeling and scenario analysis...")
        risk_prompt = f"""
        Perform risk modeling for this large loan:
        
        Analysis:
        {financial_analysis.content[:1000]}
        
        Model risk scenarios:
        - Best case
        - Base case
        - Stress case
        - Default probability
        - Expected loss
        
        Return JSON with risk assessment.
        """
        
        risk_model = call_llm(risk_prompt,
                             "You are a risk modeling specialist.",
                             model=GPT4)
        results['risk_model'] = risk_model
        print(f"✓ Complete ({risk_model.latency:.2f}s)")
        
        # Step 5: Senior review requirements
        print("[5/6] Preparing senior review package...")
        review_prep = call_llm(
            f"Prepare executive summary for credit committee review of ${loan_amount:,.0f} loan. Include all key risks and recommendations.",
            "You are preparing materials for senior credit committee.",
            model=GPT4
        )
        results['review_package'] = review_prep
        print(f"✓ Complete ({review_prep.latency:.2f}s)")
        
        # Step 6: Committee decision requirements
        print("[6/6] Generating committee decision framework...")
        committee_prep = call_llm(
            f"Create decision framework for credit committee. What questions should they ask? What additional diligence is needed?",
            "You are a credit committee advisor.",
            model=GPT4
        )
        results['committee_framework'] = committee_prep
        print(f"✓ Complete ({committee_prep.latency:.2f}s)")
        
        results['workflow_type'] = "ENHANCED_DD"
        results['processing_time'] = time.time() - start_time
        
        print(f"\nTotal processing time: {results['processing_time']:.2f}s")
        print(f"Requires: SENIOR REVIEW + COMMITTEE APPROVAL")
        print(f"{'='*80}\n")
        
        return results
    
    @classmethod
    def route_and_process(cls, application: str) -> Dict:
        """
        Main routing function: classify loan and route to appropriate workflow
        """
        print("\n" + "="*80)
        print("CONDITIONAL LOAN ROUTING SYSTEM")
        print("="*80)
        
        # Extract loan amount for routing decision
        extraction_prompt = f"""
        Extract only the loan amount from this application.
        Return as JSON: {{"loan_amount": <number>}}
        
        Application:
        {application}
        """
        
        amount_extraction = call_llm(extraction_prompt, 
                                    "You extract loan amounts.",
                                    model=GPT35)
        
        try:
            loan_amount = json.loads(amount_extraction.content)['loan_amount']
        except:
            # Fallback: try to find amount in text
            amounts = re.findall(r'\$([\d,]+)', application)
            loan_amount = float(amounts[0].replace(',', '')) if amounts else 100000
        
        # Classify
        category = cls.classify_loan(loan_amount)
        
        print(f"\nLoan Amount: ${loan_amount:,.0f}")
        print(f"Category: {category}")
        print(f"Routing to: {category} loan workflow...\n")
        
        # Route to appropriate workflow
        if category == "SMALL":
            result = cls.process_small_loan(application, loan_amount)
        elif category == "MEDIUM":
            result = cls.process_medium_loan(application, loan_amount)
        else:  # LARGE
            result = cls.process_large_loan(application, loan_amount)
        
        # Add routing metadata
        result['loan_amount'] = loan_amount
        result['category'] = category
        
        return result

In [ ]:
# Test the conditional router with different loan sizes
print("\n" + "="*80)
print("TESTING CONDITIONAL ROUTING")
print("="*80)

# Test 1: Small loan
print("\n### TEST 1: SMALL LOAN ###")
small_result = LoanRouter.route_and_process(SMALL_LOAN_APP)

# Test 2: Medium loan  
print("\n### TEST 2: MEDIUM LOAN ###")
medium_result = LoanRouter.route_and_process(SAMPLE_LOAN_APPLICATION)

# Test 3: Large loan
print("\n### TEST 3: LARGE LOAN ###")
large_result = LoanRouter.route_and_process(LARGE_LOAN_APP)

# Summary comparison
print("\n" + "="*80)
print("ROUTING EFFICIENCY COMPARISON")
print("="*80)
print(f"Small Loan:  {small_result['processing_time']:.2f}s ({small_result['workflow_type']})")
print(f"Medium Loan: {medium_result['processing_time']:.2f}s ({medium_result['workflow_type']})")
print(f"Large Loan:  {large_result['processing_time']:.2f}s ({large_result['workflow_type']})")
print("\nConditional routing optimizes processing based on risk!")
print("="*80)

---

## PART 4: PRODUCTION ORCHESTRATION (Lab 3.4)

**Duration:** 30 minutes  
**Objective:** Add production-grade features: error handling, retries, monitoring

### Challenge 4.1: Production-Ready Orchestration (30 minutes)

**Objective:** Build enterprise-grade orchestration with all production features

In [ ]:
class WorkflowStatus(Enum):
    """Workflow execution status"""
    PENDING = "pending"
    RUNNING = "running"
    COMPLETED = "completed"
    FAILED = "failed"
    PARTIAL = "partial"  # Some steps succeeded, some failed

@dataclass
class StepResult:
    """Result from individual workflow step"""
    step_name: str
    status: str  # success, failed, skipped
    response: Optional[LLMResponse] = None
    error: Optional[str] = None
    retry_count: int = 0
    start_time: float = 0.0
    end_time: float = 0.0
    
    @property
    def duration(self) -> float:
        return self.end_time - self.start_time if self.end_time > 0 else 0.0

In [ ]:
class ProductionOrchestrator:
    """
    Production-grade workflow orchestration with:
    - Error handling and retries
    - Comprehensive logging
    - Metrics tracking
    - Graceful degradation
    - Health monitoring
    """
    
    def __init__(self, max_retries: int = 3, retry_delay: float = 1.0):
        self.max_retries = max_retries
        self.retry_delay = retry_delay
        self.execution_log: List[str] = []
        self.metrics: Dict[str, Any] = {}
        
    def log(self, message: str, level: str = "INFO"):
        """Add timestamped log entry"""
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
        log_entry = f"[{timestamp}] [{level}] {message}"
        self.execution_log.append(log_entry)
        print(log_entry)
    
    def execute_step_with_retry(
        self,
        step_name: str,
        step_func: Callable,
        *args,
        **kwargs
    ) -> StepResult:
        """
        Execute a single step with retry logic
        """
        result = StepResult(
            step_name=step_name,
            status="pending",
            start_time=time.time()
        )
        
        for attempt in range(1, self.max_retries + 1):
            try:
                self.log(f"Executing {step_name} (attempt {attempt}/{self.max_retries})")
                response = step_func(*args, **kwargs)
                
                result.response = response
                result.status = "success"
                result.retry_count = attempt - 1
                result.end_time = time.time()
                
                self.log(f"✓ {step_name} completed successfully ({result.duration:.2f}s)", "SUCCESS")
                break
                
            except Exception as e:
                error_msg = str(e)
                self.log(f"✗ {step_name} failed: {error_msg}", "ERROR")
                
                if attempt < self.max_retries:
                    delay = self.retry_delay * (2 ** (attempt - 1))  # Exponential backoff
                    self.log(f"Retrying in {delay:.1f}s...", "WARNING")
                    time.sleep(delay)
                else:
                    result.status = "failed"
                    result.error = error_msg
                    result.retry_count = attempt
                    result.end_time = time.time()
        
        return result
    
    def execute_workflow(
        self,
        workflow_name: str,
        steps: List[Tuple[str, Callable, tuple, dict]],
        fail_fast: bool = False
    ) -> Dict[str, Any]:
        """
        Execute complete workflow with monitoring
        
        Args:
            workflow_name: Name of workflow
            steps: List of (step_name, function, args, kwargs) tuples
            fail_fast: Stop on first failure if True
        
        Returns:
            Complete workflow results with metrics
        """
        workflow_start = time.time()
        
        self.log("="*80)
        self.log(f"Starting workflow: {workflow_name}")
        self.log("="*80)
        
        results = {
            'workflow_name': workflow_name,
            'status': WorkflowStatus.RUNNING.value,
            'steps': {},
            'metrics': {},
            'logs': []
        }
        
        successful_steps = 0
        failed_steps = 0
        total_tokens = 0
        
        for step_name, step_func, args, kwargs in steps:
            step_result = self.execute_step_with_retry(step_name, step_func, *args, **kwargs)
            
            results['steps'][step_name] = {
                'status': step_result.status,
                'duration': step_result.duration,
                'retry_count': step_result.retry_count,
                'tokens': step_result.response.tokens if step_result.response else 0,
                'error': step_result.error
            }
            
            if step_result.status == "success":
                successful_steps += 1
                if step_result.response:
                    total_tokens += step_result.response.tokens
            else:
                failed_steps += 1
                if fail_fast:
                    self.log(f"Fail-fast enabled. Stopping workflow.", "ERROR")
                    break
        
        # Determine final status
        if failed_steps == 0:
            final_status = WorkflowStatus.COMPLETED.value
        elif successful_steps == 0:
            final_status = WorkflowStatus.FAILED.value
        else:
            final_status = WorkflowStatus.PARTIAL.value
        
        total_duration = time.time() - workflow_start
        
        results['status'] = final_status
        results['metrics'] = {
            'total_duration': total_duration,
            'successful_steps': successful_steps,
            'failed_steps': failed_steps,
            'total_steps': len(steps),
            'success_rate': successful_steps / len(steps) if steps else 0,
            'total_tokens': total_tokens
        }
        results['logs'] = self.execution_log.copy()
        
        self.log("="*80)
        self.log(f"Workflow {workflow_name} completed: {final_status}")
        self.log(f"Duration: {total_duration:.2f}s | Success: {successful_steps}/{len(steps)}")
        self.log("="*80)
        
        return results

In [ ]:
# Test production orchestrator
orchestrator = ProductionOrchestrator(max_retries=2, retry_delay=0.5)

# Define workflow steps
workflow_steps = [
    ("extract_data", step1_extract_info, (SAMPLE_LOAN_APPLICATION,), {}),
    ("verify_credit", check_credit_bureau, ({"name": "Test", "credit_score": 750},), {}),
    ("verify_employment", verify_employment, ({"employer": "TechCorp", "income": 145000},), {}),
]

# Execute workflow
production_results = orchestrator.execute_workflow(
    workflow_name="Loan Processing Pipeline",
    steps=workflow_steps,
    fail_fast=False
)

# Display results
print("\n" + "="*80)
print("WORKFLOW METRICS DASHBOARD")
print("="*80)
print(f"Status: {production_results['status']}")
print(f"Duration: {production_results['metrics']['total_duration']:.2f}s")
print(f"Success Rate: {production_results['metrics']['success_rate']*100:.1f}%")
print(f"Total Tokens: {production_results['metrics']['total_tokens']}")
print("\nStep-by-Step Results:")
for step_name, step_info in production_results['steps'].items():
    status_icon = "✓" if step_info['status'] == "success" else "✗"
    print(f"  {status_icon} {step_name}: {step_info['duration']:.2f}s "
          f"({step_info['tokens']} tokens, {step_info['retry_count']} retries)")
print("="*80)

---

## CAPSTONE CHALLENGE: ENTERPRISE LOAN PROCESSING SYSTEM

**Objective:** Build complete production system combining all patterns

In [ ]:
class EnterpriseLoanProcessor:
    """
    Complete production loan processing system combining:
    - Conditional routing
    - Parallel processing
    - Sequential workflows
    - Error handling
    - Monitoring
    """
    
    def __init__(self):
        self.orchestrator = ProductionOrchestrator(max_retries=3)
        self.router = LoanRouter()
        self.processing_stats = {
            'total_processed': 0,
            'approved': 0,
            'denied': 0,
            'review': 0,
            'total_time': 0.0,
            'total_tokens': 0
        }
    
    def process_application(self, application: str, app_id: str) -> Dict[str, Any]:
        """
        Process single loan application through complete system
        """
        start_time = time.time()
        
        print(f"\n{'='*80}")
        print(f"ENTERPRISE LOAN PROCESSOR - Application {app_id}")
        print(f"{'='*80}")
        
        # Route to appropriate workflow
        result = self.router.route_and_process(application)
        
        # Update statistics
        processing_time = time.time() - start_time
        self.processing_stats['total_processed'] += 1
        self.processing_stats['total_time'] += processing_time
        
        decision = result.get('final_decision', 'REVIEW')
        if decision == 'APPROVE':
            self.processing_stats['approved'] += 1
        elif decision == 'DENY':
            self.processing_stats['denied'] += 1
        else:
            self.processing_stats['review'] += 1
        
        return result
    
    def get_statistics(self) -> Dict[str, Any]:
        """Return processing statistics"""
        avg_time = (self.processing_stats['total_time'] / 
                   self.processing_stats['total_processed'] 
                   if self.processing_stats['total_processed'] > 0 else 0)
        
        return {
            'total_processed': self.processing_stats['total_processed'],
            'approved': self.processing_stats['approved'],
            'denied': self.processing_stats['denied'],
            'needs_review': self.processing_stats['review'],
            'average_processing_time': avg_time,
            'total_tokens': self.processing_stats['total_tokens']
        }


# Initialize enterprise processor
processor = EnterpriseLoanProcessor()

# Process multiple applications
applications = [
    (SMALL_LOAN_APP, "LA-2024-001"),
    (SAMPLE_LOAN_APPLICATION, "LA-2024-002"),
    (LARGE_LOAN_APP, "LA-2024-003")
]

results = []
for app, app_id in applications:
    result = processor.process_application(app, app_id)
    results.append(result)
    time.sleep(0.5)  # Brief pause between applications

# Display final statistics
print("\n" + "="*80)
print("ENTERPRISE PROCESSING STATISTICS")
print("="*80)
stats = processor.get_statistics()
for key, value in stats.items():
    if isinstance(value, float):
        print(f"{key}: {value:.2f}")
    else:
        print(f"{key}: {value}")
print("="*80)

---

## LAB SUMMARY

### Patterns Mastered

| Pattern | Use Case | Key Benefit | Typical Speedup |
|---------|----------|-------------|------------------|
| **Sequential** | Dependent steps | Clear logic, debuggable | 1x (baseline) |
| **Parallel** | Independent tasks | Reduced latency | 2-3x |
| **Conditional** | Different processing paths | Optimized cost/speed | Varies (2-10x for simple cases) |
| **Hybrid** | Complex workflows | Best of all patterns | 2-4x |

### Production Checklist

Before deploying orchestrated workflows:

- [ ] Error handling for each step
- [ ] Retry logic with exponential backoff
- [ ] Comprehensive logging
- [ ] Metrics collection (latency, tokens, success rate)
- [ ] Graceful degradation paths
- [ ] Health monitoring
- [ ] Cost tracking
- [ ] Performance SLAs defined
- [ ] Testing with edge cases
- [ ] Documentation

### Key Takeaways

✓ **Sequential chains** execute steps in order with clear dependencies  
✓ **Parallel processing** reduces wall-clock time significantly (~2.7x speedup)  
✓ **Conditional routing** optimizes cost and speed based on input characteristics  
✓ **Hybrid workflows** combine patterns for optimal performance  
✓ **Production features** (error handling, retries, monitoring) are essential  

---

**End of Lab 3: Prompt Flow & Orchestration**

**Next Lab:** RAG Implementation (Document Chunking, Embeddings, Vector Search)